# Stochastic Gradient Descent (SGD) Dengan Momentum Nesterov
### Modifikasi dengan Optimizer **ADAM**
Benediktus Sashenka
10117080
___

In [ ]:
from numpy import *   
import numpy as np
from numpy.random import *
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline

np.set_printoptions(precision = 3, suppress = True, formatter = {'float':'{:5.4f}'.format})

#Sigmoid function & its derivative
sigmoid = lambda Z: 1/(1+exp(-Z))
dsigmoid = lambda A: A*(1-A)

#ReLU function & its derivative
ReLU  = lambda Z: Z.clip(0)
#Derivative of ReLU function
dReLU = lambda A: (A > 0)*1

#Derivativer of tanh()
dtanh = lambda A: 1-A**2

#Derivative oh arctanh
darctanh = lambda A: 1/(A**2+1)

#Softplus function & its derivative
splus = lambda Z: log(1+exp(Z))
dsplus = lambda A: 1/(1+exp(-A))

linear = lambda X,w,b: X@w+b

"Time step (ts)"
def steps(x, step):   
    obs  = len(x)-step
    xt   = x[:obs,:]
    for i in arange(1,step+1):
        xt = hstack((xt, x[i:obs+i,:]))   
    return xt

## Modifikasi pada kelas NeuralNetwork

Pada ANN biasa, dengan *back propagation* sudah dapat diketahui nilai dari 
$\frac{\partial C}{\partial W}$, $\frac{\partial C}{\partial b}$
untuk parameter $W, b$ pada setiap hidden layer. 

Optimizer **ADAM** menyimpan $m$ dan $v$ sebagai estimator momen pertama dan kedua untuk diingat,
\begin{align}
m_t &= \beta_1  m_{t-1} + (1-\beta_1 )g_t \\
v_t &= \beta_2  v_{t-1} + (1-\beta_2 )g_t^2
\end{align}
dengan $g_t$ adalah gradien descend, $\beta_1$ dan $\beta_2$ adalah decay rates. 

Namun karena estimator $m$ dan $v$ bias, maka harus dilakukan koreksi estimator untuk menghilangkan kebiasan
\begin{equation}
\hat{m_t} = \frac{m_t}{1-\beta_1^t} ,\
\hat{v_t} = \frac{v_t}{1-\beta_2^t}
\end{equation}

Selanjutnya dapat dilakukan proses SGD dengan ADAM yaitu
\begin{equation}
\theta_{t+1} = \theta_t - \frac{\alpha}{\sqrt{\hat{v_t}}+\epsilon} \hat{m_t}
\end{equation}
dengan $\alpha$ learning rate dan $\epsilon$ untuk mencegah pembagian dengan 0. 

Ambil $\beta_1 = 0.9$, $\beta_2 = 0.999$, dan $\epsilon = 10^{-8}$, lalu dapat diterapkan proses SGD dengan ADAM untuk parameter $W$ dan $b$ sehingga

\begin{align}
m_W(t) &= \beta_1  m_W(t-1) + (1-\beta_1 )\frac{\partial C}{\partial W}, &
m_b(t) &= \beta_1  m_b(t-1) + (1-\beta_1 )\frac{\partial C}{\partial b} \\
v_W(t) &= \beta_2  v_W(t-1) + (1-\beta_2 )\left(\frac{\partial C}{\partial W}\right)^2, &
v_b(t) &= \beta_2  v_b(t-1) + (1-\beta_2 )\left(\frac{\partial C}{\partial b}\right)^2 \\
\end{align}

Lalu dilakukan koreksi estimator

\begin{align}
\hat{m_W} &= \frac{m_W(t)}{1-\beta_1^t} , & \hat{m_b} &= \frac{m_b(t)}{1-\beta_1^t} \\
\hat{v_W} &= \frac{v_W(t)}{1-\beta_2^t} , & \hat{v_b} &= \frac{v_b(t)}{1-\beta_2^t} 
\end{align}

Setelah mendapat estimator untuk momen, parameter $W$ dan $b$ dapat diperbarui
\begin{align}
W(t+1) &= W(t) - \frac{\alpha}{\sqrt{\hat{v_W}}+\epsilon} \hat{m_W} \\
b(t+1) &= b(t) - \frac{\alpha}{\sqrt{\hat{v_b}}+\epsilon} \hat{m_b} \\
\end{align}

Lalu karena sudah memakai momentum optimizer ADAM, learning rate yang digunakan tidak terlalu besar yaitu 0.001.

Catatan: kuadrat dan akar pada operasi matriks dilakukan elemen per elemen (e.g. [2, 3]^2 = [4, 9])

Referensi :
Ruder, S. 2016. An overview of gradient descent optimization algorithms.

In [ ]:
class NeuralNetwork:
    
    def __init__(self,x,y,hlayers,alpha,mbs): # ada h nodes di dalam hidden layer  hlayers = [7, 3, 34, 89]
        self.X  = x   #input
        self.y  = y   #output
        self.hlayers = hlayers   #note: untuk model Perceptron gunakan hlayers = []
        self.α  = alpha
        self.mbs = mbs
        
        #self.Xt = x   #akan digunakan untuk SGD
        #self.yt = y   #akan digunakisn di SGD
        
        assert ndim(y)            == 2     #y harus berupa matriks (berdimensi 2)
        assert type(self.hlayers) == list  #hlayers harus berupa sebuah list
        
        self.N, ni = shape(x)   #jumlah features (variables)
        self.N, no = shape(y)   #jumlah observasi (self.N) dan jumlah output (no)
        
        self.neurons = [ni]
        self.neurons.extend(self.hlayers)
        self.neurons.append(no)   #jumlah neurons per layer, termasuk input layer 
        self.nlayers  = len(self.neurons)-1  #number of layers (tidak termasuk input layer)

        "Initial values untuk parameter w and b"

        self.w, self.b = [], []
        for i in arange(self.nlayers):
            self.w.append(randn(self.neurons[i], self.neurons[i+1]))  #Sinapsis dari layer ke i menuju layer ke (i+1)  
            self.b.append(randn(1, self.neurons[i+1]))           #Bias di layer ke (i+1)    
          
        "Initial values untuk momen dengan random, momen v harus positif"
        
        self.Mw, self.Mb = [], []
        for i in arange(self.nlayers):
            self.Mw.append(randn(self.neurons[i], self.neurons[i+1]))  
            self.Mb.append(randn(1, self.neurons[i+1]))  
            
        self.Vw, self.Vb = [], []
        for i in arange(self.nlayers):
            self.Vw.append(randn(self.neurons[i], self.neurons[i+1])**2)  
            self.Vb.append(randn(1, self.neurons[i+1])**2)  

    def training(self):
        
        acak  = choice(len(self.y), int(self.mbs*len(self.y)), replace = False)   #bikin indeks pengambilan secara acak sebesar bs
        Xmini = self.X[acak]   #mini batch sample dari X
        ymini = self.y[acak]   #mini batch sample dari y
        
        epoch = 0 #menyimpan jumlah iterasi
        
        
        for Xt,yt in zip(Xmini,ymini):
        
            "Forward propagation (perambatan maju)"
    
            Z, A = [], [Xt]
        
            "Dimulai dari Hidden layers"
            for j in arange(len(hlayers)):   #hanya hidden layers
                Z.append(linear(A[j],self.w[j],self.b[j]))  #Reaksi kimia di layer ke (j + 1)
                A.append(tanh(Z[j]))     #Aliran listrik di layer ke (j + 1)
        
            "Output layer"
            Z.append(linear(A[self.nlayers-1],self.w[self.nlayers-1],self.b[self.nlayers-1]))  #Reaksi kimia di output layer
            A.append(sigmoid(Z[self.nlayers-1]))
    
            self.predicted_yt = A[self.nlayers]
            e         = yt-self.predicted_yt

            "Backward propagation (perambatan mundur)"
        
            
            "Dimulai dari output layer"
            dCdZ = [(-2*e/len(yt))*dsigmoid(A[::-1][0])]  #berbentuk list agar bisa di-append, mulai dari output layer
            
            beta = 0.1   #0.02
            "Hidden layer"                                           
            for m in arange(self.nlayers-1):  #dan mundur ke layer berikutnya, sampai hidden layer pertama
                dCdZ.append((dCdZ[m]@(self.w[::-1][m]-beta*self.Vw[::-1][m]).T)*dtanh(A[::-1][m+1]))   #delta
        
            "Perubahan parameters (w dan b):"
    
            one  = ones([1,len(yt)])
            dCdw, dCdb = [], []   #dalam bentuk list agar bisa di-append
            
            
            "_____________________ MODIFIKASI DENGAN ADAM _____________________"
            
            "Parameter ADAM"
            
            t     = epoch
            beta1 = 0.9
            beta2 = 0.999
            eps   = 1E-8
            
            "Stochastic Gradient Descents"
            
            for n in arange(self.nlayers): 
                
                "Meghitung gradien C terhadap W dan b, dengan menggunakan dC/dZ"
                
                dCdw.append(A[n].T@dCdZ[::-1][n])  
                dCdb.append(one@dCdZ[::-1][n])     
                
                "Menghitung estimator momen m dan v"
                
                self.Mw[n] = beta1*array(self.Mw[n]) + (1-beta1)*array(dCdw[n])
                self.Mb[n] = beta1*array(self.Mb[n]) + (1-beta1)*array(dCdb[n])
                
                self.Vw[n] = beta2*array(self.Vw[n]) + (1-beta2)*array(dCdw[n])**2
                self.Vb[n] = beta2*array(self.Vb[n]) + (1-beta2)*array(dCdb[n])**2
                
                "Menghilangkan kebiasan m dan v"
                
                Mw_est = array(self.Mw[n])/(1-beta1**(t+1))
                Mb_est = array(self.Mb[n])/(1-beta1**(t+1))
                Vw_est = array(self.Vw[n])/(1-beta2**(t+1))
                Vb_est = array(self.Vb[n])/(1-beta2**(t+1))
                
                "Updating parameter W dan b"
                
                self.w[n] -= (self.α/(np.sqrt(Vw_est)+eps))*Mw_est
                self.b[n] -= (self.α/(np.sqrt(Vb_est)+eps))*Mb_est
                
                
            "_____________________ MODIFIKASI DENGAN ADAM _____________________"
            
            epoch += 1
            
    
    def prediction(self, Xs, ys):   #Gunakan data secara keseluruhan
        
        "Forward propagation (perambatan maju)"
    
        Z, A = [], [Xs]
        
        "Dimulai dari Hidden layers"
        for j in arange(len(hlayers)):   #hanya hidden layers
            Z.append(linear(A[j],self.w[j],self.b[j]))  #Reaksi kimia di layer ke (j + 1)
            A.append(tanh(Z[j]))     #Aliran listrik di layer ke (j + 1)
        
        "Output layer"
        Z.append(linear(A[self.nlayers-1],self.w[self.nlayers-1],self.b[self.nlayers-1]))  #Reaksi kimia di output layer
        A.append(sigmoid(Z[self.nlayers-1]))
    
        self.predicted_y = A[self.nlayers]
        e         = ys-self.predicted_y
        self.Cost = e.T@e/len(ys)  #mean squared error

#### Prediksi harga saham ASII

In [ ]:
A  = pd.read_csv('ASII_10117080.csv')  #Data time series harian harga saham ASII
A['Adj Close'].fillna(A['Adj Close'].mean(), inplace=True)
A6 = np.array(A['Adj Close'])
B  = A6[:,newaxis]  #berupa matriks

Bmin = min(B)
Bmax = max(B)
b = (B-Bmin)/(Bmax-Bmin)

ts = 1   #Di literatur Time Series digunakan istilah 'lag' sebagai padanan istilah 'timestep' ini
xt = steps(b, ts)  #Dihasilkan matriks dengan 2 (= ts+1) kolom, kolom pertama menjadi variabel X
                   #dan kolom terkhir menjadi variabel y

x = xt[-800:,:]    #Ambil 800 observasi terakhir

#Data untuk training
Xtrain = x[0:680, :-1]   #Ambil 680 observasi yang pertama dan hilangkan kolom terakhir
ytrain = x[0:680:, -1:]  #Ambil 680 observasi yang pertama dan ambil kolom terakhir sebagai variabel y 

#Data untuk testing
Xtest = x[680:, :-1]   #ambil jumlah observasi sebanyak 120, hilangkan kolom terakhir (untuk y)
ytest = x[680:, -1:]   #ambil kolom terakhir

hlayers = [3, 2]  #Dua hidden layers, masing-masing dengan 3 neurons dan 2 neurons
alpha   = 0.001
mbs     = 0.15  #hanya 15% dari seluruh observasi yang digunakan untuk penaksiran parameter
epochs  = 500

#Rangkuman activation function di hidden layers & output layer

#tanh & sigmoid:
#SGD dengan Momentum: mbs = 0.15, alpha = 0.65, beta = 0.1, epochs = 400 initial values di luar epochs
#SGD : mbs = 1,    alpha = 0.17,  epochs = 70, 
#MBGD: mbs = 0.15, alpha = 0.165, epochs = 400, alpha = 0.27, epochs = 1600
#      mbs = 0.20, alpha = 0.165, epochs = 450

tic = datetime.now()

seed(20200219)

ann2 = NeuralNetwork(Xtrain,ytrain,hlayers,alpha,mbs)   #pembuatan object ann2

for t in range(epochs): 
    ann2.training()
    ann2.prediction(Xtrain,ytrain)   #Gunakan data untuk training secara keseluruhan
    
    if (t+1)%(epochs/10) == 0:   #tampilkan output lima kali
        print("Sampai epoch ke", t+1,"dicapai akurasi MSE sebesar %8.7f" %ann2.Cost,"dengan waktu", datetime.now()-tic)

toc = datetime.now()
print('\nAkurasi dengan data training: %8.7f' %ann2.Cost,'\n')

plt.plot(ytrain, label = 'actual series')
plt.plot(ann2.predicted_y, label = 'predicted series')
plt.legend()
plt.show()

toc = datetime.now()
print(f'Waktu yang diperlukan dari mulai training ({epochs} epochs): ', toc-tic)

#### Test model Deep Learning dengan data di periode yang lain

In [ ]:
#Parameters w0, w1 dan w2 sebagai ilmu pengetahuan atau knowledge yang diperoleh selama training 
#akan digunakan disini untuk di-test kemampuannya memprediksi. 
#Data yang dipakai Xtest dan ytest yang tidak digunakan selama training.

Cost_train = ann2.Cost    #Ini untuk data untuk training

ann2.prediction(Xtest,ytest)   #Gunakan data untuk test
predicted_ytest = ann2.predicted_y
Cost_test = ann2.Cost          #Ini dengan data untuk test 

print('\nMSE training: %8.7f'%Cost_train,'\nMSE testing : %8.7f'%Cost_test)

ytesto    = ytest*(Bmax-Bmin)+Bmin      #Kembalikan datanya ke original unit dalam satuan dolar
ytesthato = predicted_ytest*(Bmax-Bmin)+Bmin   #Kembalikan datanya ke original unit dalam satuan dolar

plt.plot(ytesto, label = 'actual series')
plt.plot(ytesthato, label = 'predicted series')
plt.legend()
plt.show()
toc = datetime.now()

print(f'Waktu untuk training (dengan {epochs} epochs) dan prediksi: ',  toc-tic)